# Load Dependencies
This notebook drives the feedback fine-tuning pipeline end to end so we can verify the service works with the sample payload in `tms-model-finetune/test.json`. Run each section sequentially.

In [2]:
# Core dependencies for the fine-tuning workflow
from __future__ import annotations

import json
import pprint
import re
from pathlib import Path

import requests

import config
from dataset import build_dataset
from schemas import FineTuneRequest
from trainer import run_finetune


# Inspect Pipeline Configuration
Load and inspect the feedback payload, fixing minor formatting issues so we can validate the detections before training.

In [4]:
# Load and sanitize the sample JSON payload
payload_path = Path("test.json")
raw_text = payload_path.read_text(encoding="utf-8")

# Remove trailing commas that make the JSON invalid
sanitized_text = re.sub(r",(?=\s*[}\]])", "", raw_text)
payload_dict = json.loads(sanitized_text)

pprint.pprint(payload_dict)
print(f"Loaded {len(payload_dict['images'])} image entries from {payload_path}")

{'images': [{'detections': [{'box': {'x_max': 250,
                                     'x_min': 120,
                                     'y_max': 200,
                                     'y_min': 80},
                             'class_id': 3},
                            {'box': {'x_max': 400,
                                     'x_min': 300,
                                     'y_max': 280,
                                     'y_min': 150},
                             'class_id': 1}],
             'image_url': 'https://res.cloudinary.com/dxqmzslkb/image/upload/v1761047784/transformers/baseline/de72eb41-12c3-43a4-9031-c2911cb986aa/baseline_de72eb41-12c3-43a4-9031-c2911cb986aa_rainy_1761047782394.jpg'}]}
Loaded 1 image entries from test.json


# Resolve Data Sources
Confirm the referenced imagery is reachable and capture basic metadata so we can troubleshoot download failures early.

In [5]:
# Probe each image URL to ensure it is reachable
for entry in payload_dict["images"]:
    url = entry["image_url"]
    try:
        response = requests.head(url, timeout=15)
        if response.status_code >= 400:
            response = requests.get(url, stream=True, timeout=15)
        size = response.headers.get("Content-Length", "unknown")
        print(f"OK: {url} (status={response.status_code}, size={size})")
    except requests.RequestException as error:
        print(f"FAILED: {url} ({error})")

OK: https://res.cloudinary.com/dxqmzslkb/image/upload/v1761047784/transformers/baseline/de72eb41-12c3-43a4-9031-c2911cb986aa/baseline_de72eb41-12c3-43a4-9031-c2911cb986aa_rainy_1761047782394.jpg (status=200, size=193420)


# Instantiate Pipeline Components
Build the `FineTuneRequest` object that mirrors the payload while overriding hyperparameters for a quick integration test.

In [6]:
# Create the pydantic request object with lightweight hyperparameters for testing
request_payload = FineTuneRequest(
    images=payload_dict["images"],
    train_replay=0,
    epochs=2,
    batch_size=4,
    learning_rate=1e-4,
    weight_decay=1e-4,
    freeze=10,
)
request_payload

FineTuneRequest(images=[FeedbackImage(image_url=HttpUrl('https://res.cloudinary.com/dxqmzslkb/image/upload/v1761047784/transformers/baseline/de72eb41-12c3-43a4-9031-c2911cb986aa/baseline_de72eb41-12c3-43a4-9031-c2911cb986aa_rainy_1761047782394.jpg', ), detections=[Detection(box=BoundingBox(x_min=120.0, y_min=80.0, x_max=250.0, y_max=200.0), class_id=3), Detection(box=BoundingBox(x_min=300.0, y_min=150.0, x_max=400.0, y_max=280.0), class_id=1)])], train_replay=0, epochs=2, batch_size=4, image_size=None, learning_rate=0.0001, weight_decay=0.0001, momentum=None, freeze=10, seed=None, device=None)

# Execute Training Pipeline
Run the fine-tuning job against the temporary dataset that mixes feedback images with the configured replay samples (disabled here for speed).

In [7]:
# Kick off the fine-tuning run (may take a few minutes depending on hardware)
summary = run_finetune(request_payload)
summary

2025-10-21 23:58:08,321 | INFO | Starting fine-tune run finetune_20251021T182808Z
2025-10-21 23:58:08,322 | INFO | Feedback images: 1
2025-10-21 23:58:08,323 | INFO | Base weights: D:\7th_sem\Software Design Project\transformer-maintenance-system\tms-fault-detection-model\weights\best.pt
2025-10-21 23:58:08,324 | INFO | Falling back to CPU
2025-10-21 23:58:08,322 | INFO | Feedback images: 1
2025-10-21 23:58:08,323 | INFO | Base weights: D:\7th_sem\Software Design Project\transformer-maintenance-system\tms-fault-detection-model\weights\best.pt
2025-10-21 23:58:08,324 | INFO | Falling back to CPU
2025-10-21 23:58:09,398 | INFO | Registered feedback sample feedback_000_baseline_de72eb41-12c3-43a4-9031-c2911cb986aa_rainy_1761047782394_b6eb874199.jpg with 2 detections
2025-10-21 23:58:09,398 | INFO | Registered feedback sample feedback_000_baseline_de72eb41-12c3-43a4-9031-c2911cb986aa_rainy_1761047782394_b6eb874199.jpg with 2 detections
2025-10-21 23:58:09,777 | INFO | Dataset ready at D:\7

TrainingSummary(run_id='finetune_20251021T182808Z', weights_path=WindowsPath('D:/7th_sem/Software Design Project/transformer-maintenance-system/tms-model-finetune/finetune_weight/best_finetune.pt'), metrics={'metrics/precision(B)': 0.6930394145729896, 'metrics/recall(B)': 0.4259259259259259, 'metrics/mAP50(B)': 0.5087361111111111, 'metrics/mAP50-95(B)': 0.26677821340388, 'fitness': 0.26677821340388}, log_path=WindowsPath('D:/7th_sem/Software Design Project/transformer-maintenance-system/tms-model-finetune/logs/finetune_20251021T182808Z.log'), dataset_size=1, feedback_samples=1, replay_samples=0, hyperparams={'epochs': 2, 'batch': 4, 'imgsz': 640, 'lr0': 0.0001, 'weight_decay': 0.0001, 'momentum': 0.937, 'freeze': 10, 'seed': 1337, 'device': 'cpu', 'train_replay': 0})

# Run Evaluation and Sanity Checks
Inspect the reported metrics and perform quick assertions so we can spot regressions early.

In [8]:
# Display metrics and assert they were produced
print(json.dumps(summary.metrics, indent=2))
assert summary.metrics, "No metrics returned by fine-tune run"
print("Metrics captured for", summary.run_id)


{
  "metrics/precision(B)": 0.6930394145729896,
  "metrics/recall(B)": 0.4259259259259259,
  "metrics/mAP50(B)": 0.5087361111111111,
  "metrics/mAP50-95(B)": 0.26677821340388,
  "fitness": 0.26677821340388
}
Metrics captured for finetune_20251021T182808Z


# Persist Artifacts and Logs
Verify that the fine-tuned weights and diagnostic artifacts are present in the expected locations.

In [9]:
# Confirm the exported weights and log files exist
weights_path = summary.weights_path
print("Weights saved to:", weights_path)
print("Exists:", weights_path.exists())
if weights_path.exists():
    print("Size (MB):", round(weights_path.stat().st_size / (1024 * 1024), 2))

log_path = summary.log_path
print("Log file:", log_path)
print("Exists:", log_path.exists())

Weights saved to: D:\7th_sem\Software Design Project\transformer-maintenance-system\tms-model-finetune\finetune_weight\best_finetune.pt
Exists: True
Size (MB): 18.29
Log file: D:\7th_sem\Software Design Project\transformer-maintenance-system\tms-model-finetune\logs\finetune_20251021T182808Z.log
Exists: True


# Automate Validation in VS Code Test Explorer
Trigger the test suite from the notebook so CI-style validation is only one cell away. Skip or adjust the command if no tests exist yet.

In [ ]:
# Run pytest from the repository root (adjust markers if needed)
import subprocess
import sys
from pathlib import Path

repo_root = Path.cwd().resolve()
if repo_root.name == "tms-model-finetune":
    repo_root = repo_root.parent

result = subprocess.run([sys.executable, "-m", "pytest", "-q"], cwd=repo_root, check=False)
print("pytest exit code:", result.returncode)
if result.returncode != 0:
    print("Tests reported failures or were skipped. Inspect output above for details.")